In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io

MAIN MISSION: See if it is possible to accuratly predict in which genre is a painting in, if we compare the pictures in certain ways (E.g. colors, contrasts, light)

TODO:
Kuidas ekstraheerida piltidest suvalised ruudud? \
Algoritm: \
    Loeme pildi sisse pikslitega\
    Loome ruudu (kui palju?)\
    Valime ruutude kaupa piltidelt pikslite väärtusi\
    Loeme sisse valitud ruudud (mis on nende kontrastid ruudu siseselt ja ruutude vahel, heledus ja tumedus, valguse kasutamine jne)\

Kuidas kasutame Skimage'it?
    

TODO: Kuidas lugeda sisse erinevat informatsiooni?\
Millist informatsiooni saame erinevatest värviväärtusest? \
Millist informatsiooni annab meile võrdlused ruutude sees ja ruutude vahel?  (Lähikontrast ja üleüldine kontrast?) 

SKIMAGE!\
\
3D numpy array:
  * y coordinate
  * x coordinate
  * RGB (in this sequence)

In [2]:
artists=pd.read_csv("artists.csv",index_col=["id"])
postimp=artists[((artists['genre'].str.contains("Post-Impressionism")==True))]
#We'll drop all the unnecessary columns
postimp.drop(columns=["years","nationality","bio","wikipedia","paintings"])

,name,genre
id,,
8,Vincent van Gogh,Post-Impressionism
28,Henri Matisse,"Impressionism,Post-Impressionism"
33,Henri de Toulouse-Lautrec,Post-Impressionism
35,Camille Pissarro,"Impressionism,Post-Impressionism"
38,Paul Cezanne,Post-Impressionism
41,Georges Seurat,Post-Impressionism
46,Paul Gauguin,"Symbolism,Post-Impressionism"


In [3]:
#reading in all the relevant paintings from the resized 
#we're using the last name
authors=[]
for author in postimp.name:
    authors.append(author.split(" ")[-1])
authors

['Gogh',
 'Matisse',
 'Toulouse-Lautrec',
 'Pissarro',
 'Cezanne',
 'Seurat',
 'Gauguin']

In [4]:
import random
import os

postimp_paintings=pd.DataFrame(columns=["Postimp","Pixeldata","Squaredata"])
for painting in os.listdir("paintings"):
    i=random.randint(0,4)
    if painting.split("_")[-2] in authors:
        new_row = pd.Series(data={"Postimp": 1, "Pixeldata": skimage.io.imread(fname="paintings/" + painting)})
        postimp_paintings=postimp_paintings.append(new_row,ignore_index=True)   
        #Add randomly every fifth painting for better training later
    elif i == 4:
        new_row = pd.Series(data={"Postimp": 0, "Pixeldata": skimage.io.imread(fname="paintings/" + painting)})
        postimp_paintings=postimp_paintings.append(new_row,ignore_index=True)

print(len(postimp_paintings))
print(postimp_paintings.head())

3053
  Postimp                                          Pixeldata Squaredata
0       0  [[199, 201, 202, 195, 184, 181, 194, 209, 204,...        NaN
1       0  [[192, 192, 199, 211, 215, 202, 182, 167, 162,...        NaN
2       0  [[220, 214, 212, 214, 210, 203, 203, 209, 211,...        NaN
3       0  [[239, 239, 241, 245, 247, 247, 243, 239, 236,...        NaN
4       0  [[241, 239, 239, 243, 244, 242, 240, 240, 241,...        NaN


In [5]:
#Jagame pildi gridiks sellepärast, et saaksime adekvaatselt valida küll suvaliselt ruute sellest gridist, aga et need ei korduks. Vot
#Märgime ära gridi kordinaatide ühised nimetajad, mille järgi saab pildil navigeerida (nt (0,0), mis on esimene ruut vasakul üleval nurgas)
#Kasutame piltide resized versioone, hoida kokku andmete pealt
import random
squaredata =pd.DataFrame(columns=["Squaredata"])
for data in postimp_paintings["Pixeldata"]:
    used_squares=[]
    while len(used_squares)<25:
        y = int(len(data)/20)
        x = int(len(data[0])/20)
        rand_y=random.randint(0,19)
        rand_x=random.randint(0,19)
        if (rand_y,rand_x) not in used_squares:
            used_squares.append((rand_y,rand_x))
            #Now we have the random starting points to the squares which we can use
    squares_with_info=[]
    for start in used_squares:
        for i in range(20):
            one_row_of_pixels=[]
            for j in range(20):
                if not ((start[0]*y+i >= len(data))) | (start[1]*x+j >= len(data[0])):
                    try: 
                        one_row_of_pixels.append(((data[start[0]*y+i][start[1]*x+j][0]),(data[start[0]*y+i][start[1]*x+j][1]),(data[start[0]*y+i][start[1]*x+j][2])))
                    except:
                        continue
            squares_with_info.append(one_row_of_pixels)
    if not (squares_with_info == "nan") | (squares_with_info[0] == []):
        squaredata=squaredata.append({"Squaredata": squares_with_info},ignore_index=True)
postimp_paintings["Squaredata"] = squaredata
postimp_paintings.drop(columns=["Pixeldata"],inplace=True)
postimp_paintings.dropna(inplace=True)
#since we experience some data loss (some paintings are read in in a different manner), we need to drop the remaining rows with NaN values
#squares_with_info is a 2D array, because it's not important for us to know where the square was, but which pixels were in one square


In [6]:
data = postimp_paintings.copy
data.rename(columns={"Postimp": "Post-Impressionism"})
#df['c'] = df.apply(lambda row: row.a + row.b, axis=1)
#data = pd.DataFrame(columns=["Post-Impressionism","Bright_light","Bright_med","Bright_dark",
#                            "Color_light","Color_med","Color_dark",
#                           "Colorsq_light","Colorsq_med","Colorsq_dark",
#                           "Grey_light","Grey_med","Grey_dark"])


AttributeError: 'function' object has no attribute 'rename'

In [ ]:
from math import sqrt

#data used in each function is a 2D dataframe which describes the values of the squares selected from the paintings
def brightness(square):
    bright_average = 0
    bright_values = []
    for rgb in square:
        brigth_sum = 0
        for n in rgb:
            bright_sum += math.sqrt(0.299 * (n[0] * n[0]) + 0.587 * (n[1] * n[1]) + 0.114 * (n[2] * n[2]))
        bright_values.append(round(bright_sum / len(rgb)))
    for value in bright_values:
        bright_average += value
    bright_average=bright_average/len(bright_values)
    if bright_average >= 170:
        return 'light'
    elif bright_average > 85:
        return 'medium'
    else:
        return 'dark'

def brightness0(r,g,b):
    brightness = math.sqrt(0.299 * (r * r) + 0.587 * (g * g) + 0.114 * (b * b))
    if  brightness > 170:
        return 'light'
    elif brightness > 85:
        return 'medium'
    else:
        return 'dark'
    
#see funktsioon otsib üles keskmise värvi, aga võtab iga rgb osa ruutu ja siis lõpus leiab ruutjuure
#this function finds the "average color" of the square
def color_sq(square):
    average_colors_sq = []
    for rgb in square:
        r = 0
        g = 0
        b = 0
        for n in rgb:
            r += n[0]*n[0]
            g += n[1]*n[1]
            b += n[2]*n[2]
        r = round(sqrt(r/len(rgb)))
        g = round(sqrt(g/len(rgb)))
        b = round(sqrt(b/len(rgb)))
        average_colors_sq.append((r,g,b))
    r=0
    g=0
    b=0
    for rgb in average_colors_sq:
        r += rgb[0]
        b += rgb[1]
        g += rgb[2]
    return brightness0((r/len(average_colors_sq),g/len(average_colors_sq),b/len(average_colors_sq)))

#see funktsioon otsib üles keskmise värvi, aga ei võta ruutu
def color(square):
    average_colors_sq = []
    for rgb in square:
        r = 0
        g = 0
        b = 0
        for n in rgb:
            r += n[0]
            g += n[1]
            b += n[2]
        r = round(sqrt(r/len(rgb)))
        g = round(sqrt(g/len(rgb)))
        b = round(sqrt(b/len(rgb)))
        average_colors_sq.append((r,g,b))
    r=0
    g=0
    b=0
    for rgb in average_colors_sq:
        r += rgb[0]
        b += rgb[1]
        g += rgb[2]
    return brightness0((r/len(average_colors_sq),g/len(average_colors_sq),b/len(average_colors_sq)))

def distance(rgb, colors):
    for i in range(len(rgb)-1):
        distance = round(sqrt(pow((rgb[i][0] - rgb[i+1][0]),2) + pow((rgb[i][1] - rgb[i+1][1]),2) + pow((rgb[i][2] - rgb[i+1][2]),2)))
        if distance > 50 and rgb[i] not in colors:
            colors.append(rgb[i])
            print(colors)
    if len(distance(colors, [])) != len(colors):
        distance(colors, [])
    else:
        return colors
    
            
                                  
distance([[5,5,5],[111,111,111],[0,0,0],[5,5,5],[111,111,111],[0,0,0],[2,4,333],[110,2,111],[203,0,222]], [])

#def rgb2gray(square):
#    grayscaled_square = []
#    for rgb in square:
#       grayscaled_row = []
#       for n in rgb:
#           grayscaled.append((n[0] * 0.2989, n[1] * 0.5870, n[2] * 0.1140))
#       grayscaled_square.append(grayscaled_row)
#   return brightness(grayscaled)



In [ ]:
#Fill out the data dataframe, which we will later use for model training
data
